In [151]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import matplotlib
import theano
import theano.tensor as T
import pydot
from IPython.display import Image
from IPython.display import SVG
import timeit

# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [178]:
import h5py

file_name = 'data_theano.h5'

def save_model(model):
#     a = np.random.random(size=(100,20))
#     h5f.close()
    h5f = h5py.File(file_name, 'w')
    adict=dict(W1=model['W1'], b1=model['b1'], W2=model['W2'], b2=model['b2'])
    for k,v in adict.items():
        h5f.create_dataset(k,data=v)

#     h5f.create_dataset('dataset_1', data=model)
    h5f.close()

def load_model():
    h5f = h5py.File(file_name,'r')
    model = {}
    W1.set_value(h5f['W1'][:])
    b1.set_value(h5f['b1'][:])
    W2.set_value(h5f['W2'][:])
    b2.set_value(h5f['b2'][:])
#     model['W1'] = h5f['W1'][:]
#     model['b1'] = h5f['b1'][:]
#     model['W2'] = h5f['W2'][:]
#     model['b2'] = h5f['b2'][:]
    h5f.close()
    return model

In [153]:
with open('../../data/SlovarIJS_BESEDE_utf8.lex') as f:
    content = f.readlines()
content = [x.decode('utf8').split('\t') for x in content]

In [154]:
# CREATE dictionary AND max_word
accetuated_vowels = [u'à', u'á', u'ä', u'é', u'ë', u'ì', u'í', u'î', u'ó', u'ô', u'ö', u'ú', u'ü']
default_vowels = [u'a', u'e', u'i', u'o', u'u']
vowels = []
vowels.extend(accetuated_vowels)
vowels.extend(default_vowels)
def is_vowel(word_list, position):
    if word_list[position] in vowels:
        return True
    if word_list[position] == u'r' and \
    (position - 1 < 0 or word_list[position - 1] not in vowels) and \
    (position + 1 >= len(word_list) or word_list[position + 1] not in vowels):
        return True
    return False
        
    

dictionary = ['']
line = 0
max_word = 0
# ADD 'EMPTY' VOWEL
max_num_vowels = 0
for el in content:
    num_vowels = 0
    i = 0
    try: 
        if len(el[3]) > max_word:
            max_word = len(el[3])
#             print line
        if len(el[0]) > max_word:
            max_word = len(el[0])
            print line
        for c in list(el[3]):
            if is_vowel(list(el[3]), i):
                num_vowels += 1
            if c not in dictionary:
                dictionary.append(c)
            i += 1
        for c in list(el[0]):
            if c not in dictionary:
                dictionary.append(c)
        if num_vowels > max_num_vowels:
            max_num_vowels = num_vowels
#             print line
    except Exception, e:
        print line - 1
        print el
    line += 1
dictionary = sorted(dictionary)
max_num_vowels += 1

# for e in dictionary:
#     print e

In [155]:
# GENERATE X and y
def generate_presentable_y(accetuations_list, word_list):
#     print accetuations_list
#     print word_list
    while len(accetuations_list) < 2:
        accetuations_list.append(0)
    if len(accetuations_list) > 2:
        accetuations_list = accetuations_list[:2]
#     empty has to be positive
    accetuations_list = np.array(accetuations_list)
#     accetuations_list += 1
    final_position = accetuations_list[0] + max_num_vowels * accetuations_list[1]
    return final_position
    

X = np.zeros((len(content), max_word*len(dictionary)))
y = np.zeros((len(content), max_num_vowels * max_num_vowels ))


i = 0
for el in content:
    j = 0
    for c in list(el[0]):
        index = 0
        for d in dictionary:
            if c == d:
                X[i][index + j * max_word] = 1
                break
            index += 1
        j += 1
    j = 0
    word_accetuations = []
    num_vowels = 0
    for c in list(el[3]):
        index = 0
#         in_dict = False
        if is_vowel(el[3], j):
            num_vowels += 1
        for d in accetuated_vowels:
#             print c
#             print d
            if c == d:
#                 print 'HERE!'
                word_accetuations.append(num_vowels)
#                 y[j * len(dictionary) + index][i] = 1
#                 in_dict = True
                break
            index += 1
#         if not in_dict:
#             print 'NOOOOO!!!'
#             print el
#             print i
        j += 1
#     print len(y.T[i])
#     print sum(y.T[i])
    
#     y = y.T
    y[i][generate_presentable_y(word_accetuations, list(el[3]))] = 1
#     print '---------------------------------'
#     for a in y[i]:
#         print a
#     print sum(y[i])
#     print y[:][i]
#     if i > 100:
#         break
    i += 1


In [156]:
def decode_position(y):
    max_el = 0
    i = 0
    pos = -1
#     pos = y.argmax(axis=0)
    for el in y:
#         print el
        if el > max_el:
            
            max_el = el
#             print 'HERE!!!'
#             print pos
            pos = i
        i += 1
    return [pos % max_num_vowels, pos / max_num_vowels]
    
decode_position(y[157303])
# decode_position(y[157241])
# print y[2]

[1, 3]

In [157]:
X = X[:100000]
y = y[:100000]
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
X, y = unison_shuffled_copies(X, y)
# print X.shape

train_X = X.astype(np.float32)
train_y = y.astype(np.int32)

num_examples = len(X) # training set size
nn_input_dim = max_word * len(dictionary) # input layer dimensionality
nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_hdim = 500

# Gradient descent parameters (I picked these by hand)
epsilon = 1 # learning rate for gradient descent
reg_lambda = 1 # regularization strength

In [158]:
train_y.shape

# Our data vectors
X = T.matrix('X') # matrix of doubles
y = T.lmatrix('y') # vector of doubles

# Shared variables with initial values. We need to learn these.
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim), name='W1')
b1 = theano.shared(np.zeros(nn_hdim), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim), name='b2')

In [159]:
# Forward propagation
# Note: We are just defining the expressions, nothing is evaluated here!
z1 = X.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
y_hat = T.nnet.softmax(z2) # output probabilties

# The regularization term (optional)
loss_reg = 1./num_examples * reg_lambda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2))) 
# the loss function we want to optimize
loss = T.nnet.categorical_crossentropy(y_hat, y).mean() + loss_reg
# loss = T.nnet.binary_crossentropy(y_hat, y).mean() + loss_reg


# Returns a class prediction
prediction = T.argmax(y_hat, axis=1)

In [160]:
# Theano functions that can be called from our Python code
forward_prop = theano.function([X], y_hat)
calculate_loss = theano.function([X, y], loss)
predict = theano.function([X], prediction)

# Example call: Forward Propagation
# forward_prop([[1,2]])

In [161]:
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)

In [162]:
gradient_step = theano.function(
    [X, y],
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

In [176]:
# This function learns parameters for the neural network and returns the model.
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(num_passes=125, print_loss=False):
    
    # Re-Initialize the parameters to random values. We need to learn these.
    # (Needed in case we call this function multiple times)
    np.random.seed(0)
    W1.set_value(np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim))
    b1.set_value(np.zeros(nn_hdim))
    W2.set_value(np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim))
    b2.set_value(np.zeros(nn_output_dim))
    
    # Gradient descent. For each batch...
    for i in xrange(0, num_passes):
#         print type(W1)
        save_model({'W1': W1.get_value(borrow=True), 'b1': b1.get_value(borrow=True), 'W2': W2.get_value(borrow=True), 'b2': b2.get_value(borrow=True)})
        # This will update our parameters W2, b2, W1 and b1!
        print 'NUM: ' + str(i)
        gradient_step(train_X, train_y)
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 25 == 0:
            print "Loss after iteration %i: %f" %(i, calculate_loss(train_X, train_y))

In [164]:
print type(train_y[0][0])
print train_X[:3]
print train_X.shape
print train_y.shape
print train_y[:3]

<type 'numpy.int32'>
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(100000, 989)
(100000, 121)
[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0]]


In [177]:
# Build a model with a 3-dimensional hidden layer
build_model(print_loss=True)

# Plot the decision boundary
plot_decision_boundary(lambda x: predict(x))
plt.title("Decision Boundary for hidden layer size 3")

NUM: 0
Loss after iteration 0: 3.883366
NUM: 1
NUM: 2
NUM: 3
NUM: 4
NUM: 5
NUM: 6
NUM: 7
NUM: 8
NUM: 9
NUM: 10
NUM: 11
NUM: 12
NUM: 13
NUM: 14
NUM: 15
NUM: 16
NUM: 17
NUM: 18
NUM: 19
NUM: 20
NUM: 21
NUM: 22
NUM: 23
NUM: 24
NUM: 25
Loss after iteration 25: 1.222089
NUM: 26
NUM: 27
NUM: 28
NUM: 29
NUM: 30
NUM: 31
NUM: 32
NUM: 33
NUM: 34
NUM: 35
NUM: 36
NUM: 37
NUM: 38
NUM: 39
NUM: 40
NUM: 41
NUM: 42
NUM: 43
NUM: 44
NUM: 45
NUM: 46
NUM: 47
NUM: 48
NUM: 49
NUM: 50
Loss after iteration 50: 0.897985
NUM: 51
NUM: 52
NUM: 53
NUM: 54
NUM: 55
NUM: 56
NUM: 57
NUM: 58
NUM: 59
NUM: 60


KeyboardInterrupt: 

In [190]:
predict([generate_input_from_word('testiram')])[0]

2

In [24]:
def generate_input_from_word(word):
    x = np.zeros(max_word*len(dictionary))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[index + j * max_word] = 1
                break
            index += 1
        j += 1
    return x

model = load_model()
prediction = predict(model, generate_input_from_word('hidrija'))
print decode_position(prediction[0])

IOError: Unable to open file (Unable to open file: name = 'data_theano.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)